In [1]:
import torch
import torch.nn as nn

#### **Simplified Self-Attention**


In [2]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

**Single Input Attention Calculation**


In [8]:
query = inputs[1] # Getting the second row, or the features of the token "journey"

attn_scores_2 = torch.empty(inputs.shape[0])

# loop through the tokens in the sequence
for i, x_i in enumerate(inputs):
    # take the dot product of each token embedding vector
    attn_scores_2[i] = torch.dot(x_i, query)
    
    # Dot Product example:
    # torch.Tensor([1,2,3,4,5]).dot(torch.Tensor([2,1,2,1,2]))
    # (1*2)+(2*1)+(3*2)+(4*1)+(5*2) = tensor(24.)

attn_scores_2


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [19]:
# Normalize the attention scores
attn_scores_norm = attn_scores_2 / torch.sum(attn_scores_2)
print(f"Unnormailzed: {attn_scores_2}")
print(f"Normailzed: {attn_scores_norm}")

Unnormailzed: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Normailzed: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


In [30]:
# But softmax is more desireable for normalization
# Do note that there are underflow and overflow issues that come from this softmax implementation
print(attn_scores_2.exp() / attn_scores_2.exp().sum())

# This softmax implementation is preffered.
print(attn_scores_2.softmax(dim=0))

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [46]:
# calculate the updated embeddings
inputs.T @ attn_scores_2.softmax(dim=0)

tensor([0.4419, 0.6515, 0.5683])

**Full Attention Calculation**


In [100]:
# Get the attention scores for the query
attn_scores = inputs @ inputs.T

# normalize w/ softmax
attn_weights = attn_scores.softmax(dim=-1)

In [101]:
# Compute vectors
attn_weights @ inputs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

#### **Self-Attention**


In [103]:
x_2 = inputs[1] # second input token
d_in = inputs.shape[1] # The input embedding size, d=3
d_out = 2 # the output embedding size

In [133]:
torch.manual_seed(42)
# We will use three matricies to project the embedded tokens, into:
# query vector: What we are "interested in"
W_query = nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
# key vector: What we have
W_key = nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
# Value vector: what information to communicate if it is "interesting"
W_value = nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)

In [134]:
query = inputs @ W_query
keys = inputs @ W_key
values = inputs @ W_value

In [135]:
qk_attn = query @ keys.T

In [136]:
# using the key dimension
d_k = keys.shape[1]
# divide by sqrt if embedding dimension for scalling
# And apply softmax
qk_norm = (qk_attn * d_k**-0.5).softmax(dim=-1)

In [138]:
qk_norm

tensor([[0.1596, 0.1217, 0.1202, 0.2238, 0.1343, 0.2405],
        [0.1686, 0.1487, 0.1473, 0.1899, 0.1408, 0.2047],
        [0.1688, 0.1501, 0.1487, 0.1882, 0.1417, 0.2024],
        [0.1686, 0.1590, 0.1581, 0.1772, 0.1523, 0.1848],
        [0.1690, 0.1798, 0.1798, 0.1526, 0.1645, 0.1543],
        [0.1670, 0.1448, 0.1435, 0.1945, 0.1422, 0.2080]])

In [137]:
qk_norm @ values

tensor([[0.5141, 0.3639],
        [0.5633, 0.3251],
        [0.5659, 0.3221],
        [0.5839, 0.2941],
        [0.6180, 0.2539],
        [0.5575, 0.3262]])

**Putting it all together**


In [150]:
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.randn(d_in, d_out))
        self.W_key = nn.Parameter(torch.randn(d_in, d_out))
        self.W_value = nn.Parameter(torch.randn(d_in, d_out))
    
    def forward(self, x:torch.Tensor):
        q = x @ self.W_query
        k = x @ self.W_key
        v = x @ self.W_value

        # multiply query and keys
        attn_scores = q @ k.T

        # Scaled normalization
        d_k = k.shape[1]
        attn_weights = (attn_scores * d_k**-0.5).softmax(dim=-1)

        # qk normalized matmul values to get output
        out = attn_weights @ v
        return out

In [151]:
torch.manual_seed(42)
self_attn = SelfAttention_v1(3, 2)

self_attn(inputs)

tensor([[0.5141, 0.3639],
        [0.5633, 0.3251],
        [0.5659, 0.3221],
        [0.5839, 0.2941],
        [0.6180, 0.2539],
        [0.5575, 0.3262]], grad_fn=<MmBackward0>)

In [154]:
class SelfAttention_v2(nn.Module):
    """Implementation using nn.Linear instead of matrix multiplication"""
    def __init__(self, d_in, d_out, bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=bias)
        self.W_keys = nn.Linear(d_in, d_out, bias=bias)
        self.W_value = nn.Linear(d_in, d_out, bias=bias)
    
    def forward(self, x:torch.Tensor):
        q = self.W_query(x)
        k = self.W_keys(x)
        v = self.W_value(x)

        # mul q & k
        attn_scores = q @ k.T
        
        # normalize
        attn_weights = (attn_scores * k.shape[1]**-0.5).softmax(dim=-1)

        out = attn_weights @ v
        return out

In [155]:
torch.manual_seed(42)
attn_v2 = SelfAttention_v2(3, 2)
attn_v2(inputs)

tensor([[0.3755, 0.2777],
        [0.3761, 0.2831],
        [0.3761, 0.2833],
        [0.3768, 0.2763],
        [0.3754, 0.2836],
        [0.3772, 0.2746]], grad_fn=<MmBackward0>)

#### **Causal Attention**


In [186]:
class CausalSelfAttention(nn.Module):
    """Decoder Only Attention"""
    def __init__(self, d_in:int, d_out:int, bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=bias)
        self.W_key = nn.Linear(d_in, d_out, bias=bias)
        self.W_value = nn.Linear(d_in, d_out, bias=bias)
    
    def forward(self, x:torch.Tensor):
        q: torch.Tensor = self.W_query(x)
        k: torch.Tensor = self.W_key(x)
        v: torch.Tensor = self.W_value(x)

        # compute attn scores
        attn_scores = q @ k.T

        # Since we can only attend to what is previously shown
        attn_mask = ~torch.ones(attn_scores.shape).tril().bool()
        # We need to now mask with (-inf) so softmax only deals with what we have
        masked_attn = attn_scores.masked_fill(attn_mask, -torch.inf)
        # Now scaled norm
        attn_weights = (masked_attn * k.shape[1]**-0.5).softmax(dim=-1)

        out = attn_weights @ v
        return out




In [187]:
torch.manual_seed(42)
attn_v3 = CausalSelfAttention(3,2)
attn_v3(inputs)


tensor([[0.4429, 0.1077],
        [0.4656, 0.2597],
        [0.4732, 0.3030],
        [0.4135, 0.2921],
        [0.4078, 0.2567],
        [0.3772, 0.2746]], grad_fn=<MmBackward0>)

**Add in Dropout & Make it more compact**


In [255]:
class CausalAttention(nn.Module):
    """Decoder Only Attention"""
    def __init__(self, d_in:int, d_out:int, context_length:int, dropout:float, bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=bias)
        self.W_key = nn.Linear(d_in, d_out, bias=bias)
        self.W_value = nn.Linear(d_in, d_out, bias=bias)
        self.dropout = nn.Dropout(dropout) 
        # this is really a mask but GPT-2 and hf have buffer
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x:torch.Tensor):
        # create dimensions
        _, num_tokens, _ = x.shape # (B, tokens, )
        # print(num_tokens)
        q: torch.Tensor = self.W_query(x)
        k: torch.Tensor = self.W_key(x)
        v: torch.Tensor = self.W_value(x)

        # compute attn scores
        attn_scores = q @ k.transpose(1,2)

        # New, _ ops are in-place
        # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  

        attn_weights = torch.softmax(attn_scores * keys.size(-1)**-0.5, dim=-1)

        attn_weights = self.dropout(attn_weights)

        out = attn_weights @ v
        return out


In [251]:
# set an example number of tokens
token_count = 6 
inputs = torch.randn((token_count, 3))
print(inputs)

batch = torch.stack((inputs, inputs), dim=0)
batch.shape

tensor([[-0.3885, -0.9343,  1.0533],
        [ 0.1388, -0.2044, -2.2685],
        [-0.9133, -0.4204,  1.3111],
        [-0.2199,  0.1838,  0.2293],
        [ 0.6177, -0.2876,  0.8218],
        [ 0.1512,  0.1036, -2.1996]])


torch.Size([2, 6, 3])

In [252]:
torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

6
tensor([[[ 0.2944,  0.8538],
         [ 0.4221, -0.2458],
         [ 0.3523,  0.3993],
         [ 0.2727,  0.2175],
         [ 0.0894,  0.3412],
         [ 0.2619, -0.1882]],

        [[ 0.2944,  0.8538],
         [ 0.4221, -0.2458],
         [ 0.3523,  0.3993],
         [ 0.2727,  0.2175],
         [ 0.0894,  0.3412],
         [ 0.2619, -0.1882]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


#### **Multi-head Attention**

Adding multiple heads is essentially stacking multiple layers of single attention heads.
This creates multiple latent spaces, so the model can attend to different to different information.


In [256]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in:int, d_out:int, context_length:int, dropout:float, num_heads=1, bias=False):
        super().__init__()

        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, bias) for _ in range(num_heads)]
        )
    
    def forward(self, x:torch.Tensor):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [257]:
# set an example number of tokens
token_count = 6
inputs = torch.randn((token_count, 3))
print(inputs)

batch = torch.stack((inputs, inputs), dim=0)
batch.shape

tensor([[-0.1690,  0.9178, -0.3885],
        [-0.9343, -0.4991, -1.0867],
        [ 0.9624,  0.2492, -0.4845],
        [-2.0929,  0.0983, -0.0935],
        [ 0.2662, -0.5850, -0.3430],
        [-0.6821, -0.9887, -1.7018]])


torch.Size([2, 6, 3])

In [269]:
torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = inputs.size(-1), 4
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=4
)

context_vecs = mha(batch)

print(context_vecs[0,:,:])
print("context_vecs.shape:", context_vecs.shape)

tensor([[ 0.3116,  0.5603,  0.0720,  0.4299,  0.6344, -0.1544,  0.1832,  0.2823,
          0.0813,  0.5999, -0.2545, -0.1395, -0.3446,  0.2793, -0.1714, -0.0352],
        [ 0.3746,  0.2453, -0.4253,  0.0037,  0.5332,  0.0141,  0.3457, -0.0426,
         -0.5745,  0.4919,  0.0050, -0.2978,  0.0199,  0.2265, -0.0673, -0.2861],
        [ 0.2100,  0.4458, -0.1023,  0.1582,  0.4404,  0.0066,  0.4027, -0.2082,
         -0.0324,  0.3846, -0.0236, -0.3122, -0.0088,  0.0382, -0.0439, -0.3194],
        [ 0.3731, -0.0134, -0.5013, -0.0939,  0.4094, -0.0367,  0.2778, -0.0520,
         -0.6470,  0.3848, -0.0049, -0.1563, -0.0135,  0.2293, -0.0666, -0.1699],
        [ 0.2431,  0.0763, -0.3272, -0.0530,  0.2710,  0.0839,  0.2938, -0.1943,
         -0.3923,  0.2629,  0.0495, -0.2164,  0.0562,  0.1424, -0.0290, -0.1906],
        [ 0.3435, -0.0360, -0.5659, -0.1846,  0.3624, -0.0688,  0.2670, -0.0778,
         -0.4533,  0.2617,  0.1422, -0.3615,  0.1821,  0.1153, -0.0007, -0.3660]],
       grad_fn=<Slice

**Adding Weight Splits**

Instead of creating a module list of attention heads, we instead created one MultiHeaded Attention implementation,
where we define one q,k & v weight matrix, and split then to obtain the separate heads.


##### **To better understand this, lets do two things**

1. Initialize all of the layers, dimensions, etc in a class.
2. Step through the forward pass peice by peice.


In [314]:
class MultiHeadAttentionEX(nn.Module):

    def __init__(self, d_in:int, d_out:int, context_length:int, dropout:float, num_heads=1, bias=False):
        super().__init__()
        assert(d_out % num_heads == 0), "The output dimension must be divisible by the number of heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # This reduces the projection dimention to match the desired output.

        # Define q, k, v. Remember these...
        self.W_query = nn.Linear(d_in, d_out, bias=bias)
        self.W_key = nn.Linear(d_in, d_out, bias=bias)
        self.W_value = nn.Linear(d_in, d_out, bias=bias)

        self.o_proj = nn.Linear(d_out, d_out) # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)

        # Create mask for decoder block
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

In [352]:
inputs = torch.randn((6,5))

In [353]:
# Create the batch inputs 
x_batch = torch.stack([inputs, inputs], dim=0)
print(f"Batch Shape: {x_batch.shape}")
print(f"There are {x_batch.shape[0]} exmaples of size {x_batch.shape[1]}x{x_batch.shape[2]} in the batch")

Batch Shape: torch.Size([2, 6, 5])
There are 2 exmaples of size 6x5 in the batch


In [354]:
batch_size, context_length, d_in = x_batch.shape
d_out = 2

mha = MultiHeadAttentionEX(d_in, d_out, context_length, 0.0, num_heads=2)

_start of the forward pass_


In [355]:
# forward(x): Think of the forward pass starting here.
x = x_batch

b, num_tokens, d_in = x.shape

# create the full k, q, v matricies
keys = mha.W_key(x) # Shape: (b, num_tokens, d_out)
queries = mha.W_query(x)
values = mha.W_value(x)

_split the matricies using `.view()`_


In [356]:
print(f"Previous key dimension: {keys.shape}")
keys = keys.view(b, num_tokens, mha.num_heads, mha.head_dim)
print(f"Post split key dimension: {keys.shape}")

Previous key dimension: torch.Size([2, 6, 2])
Post split key dimension: torch.Size([2, 6, 2, 1])


**So what did we do here?**

Using .view() a way to take an exisitng tensor and temporarily hold it in newly defined dimensions, we split the output dimension by the number of heads we have.

_Example:_

- Output dimension = 16 | Attention heads 2 | Input dimensions are 2x6x16 (batch, context_length, d_out)
- New head dimension is 16 // 2 => 8 (Will explain // shortly)
- So we now want the key matrix to look like this 2x6x(attn_heads)x(new_dim) => 2x6x2x8

- So you can think of treating the last two dimesnions as "one".

- What happens if we increase the number of attention heads to 4?
- New head dimension is 16 // 4 => 4 (will explain // shortly)
- So we have a key matric like 2x6x4x4 (Note how the last two dimensions == Output Dimension)


**Let's rework the example using the example numbers**


Let's define our example inputs to have 8 tokens (context_length of 8) and have an input dimension of 6


In [391]:
tokens, in_dim = 8, 6
inputs_v2 = torch.randn((tokens, in_dim))
# Create the batch inputs 
x_batch = torch.stack([inputs_v2, inputs_v2], dim=0)
print(f"Batch Shape: {x_batch.shape}")
print(f"There are {x_batch.shape[0]} exmaples of size {x_batch.shape[1]}x{x_batch.shape[2]} in the batch")
batch_size, context_length, d_in = x_batch.shape
d_out, num_heads = 16, 4

Batch Shape: torch.Size([2, 8, 6])
There are 2 exmaples of size 8x6 in the batch


In [392]:
batch_size, context_length, d_in = x_batch.shape
d_out, num_heads = 16, 4

mha = MultiHeadAttentionEX(d_in, d_out, context_length, 0.05, num_heads=4)

In [393]:
# forward(x): Think of the forward pass starting here.
x = x_batch

b, num_tokens, d_in = x.shape

# create the full k, q, v matricies
keys:torch.Tensor = mha.W_key(x) # Shape: (b, num_tokens, d_out)
queries:torch.Tensor = mha.W_query(x)
values:torch.Tensor = mha.W_value(x)

_split the matricies_


In [394]:
print(f"Previous key dimension: {keys.shape}")
keys = keys.view(b, num_tokens, mha.num_heads, mha.head_dim)
print(f"Post split key dimension: {keys.shape}")

Previous key dimension: torch.Size([2, 8, 16])
Post split key dimension: torch.Size([2, 8, 4, 4])


_do the same for the query and the values_


In [395]:
queries = queries.view(b, num_tokens, mha.num_heads, mha.head_dim)
values = values.view(b, num_tokens, mha.num_heads, mha.head_dim)

_Now we need to Transpose the key matrix_

Wait why are we doing this now?

- Our initial dimensions were (batch, context_length, number of attn heads, dim of the heads)
- We want to swap the context length with the number of attn, heads so:
  - (batch, number of attn heads, context_length, dim of the heads)
- If you cannot see why this is happening now, we will cover it in a second.


In [396]:
keys = keys.transpose(1, 2)
queries = queries.transpose(1, 2)
values = values.transpose(1, 2)

So the reason we need to take the transpose is so we compute the attn scores, and allow us to take advantage of PyTorch's broadcasting functionality. [What is broadcasting?](https://pytorch.org/docs/stable/notes/broadcasting.html)


In [397]:
queries.shape, keys.transpose(2,3).shape

(torch.Size([2, 4, 8, 4]), torch.Size([2, 4, 4, 8]))

now we are going to compute the attention scores like we did previosly with matrix mulitplication. Note that we are broadcasting the multiplication across the first two dimensions, so realy we are doing 8x4 (queries) and 4x8 (keys) along the two dimensions.


In [398]:
attn_scores = queries @ keys.transpose(2,3)

In [399]:
attn_scores.shape

torch.Size([2, 4, 8, 8])

Just like before, we need to normalize the attn_scores into attention weights. As you can see the matricies are not normalized


In [400]:
attn_scores[0,0,:,:]

tensor([[-0.6037, -0.3267, -0.8748,  0.0127, -0.2828,  0.1720,  0.0934,  0.4857],
        [-0.9395, -1.3695, -0.0316, -0.3506, -0.2686, -0.3295,  0.8011, -0.5893],
        [-0.0748,  0.0079,  0.6271, -0.0506,  0.2189, -0.0906,  0.0236, -0.0896],
        [-0.1102, -0.4110,  0.4088, -0.1569, -0.0359, -0.1355,  0.3467, -0.2141],
        [-0.4095, -0.7016,  0.1857, -0.1982, -0.0505, -0.2666,  0.3988, -0.5665],
        [-0.0080, -0.1904,  0.6200, -0.1120,  0.1329, -0.1724,  0.1757, -0.3298],
        [ 0.5833,  1.0433,  0.1107,  0.2706,  0.3087,  0.1986, -0.6728,  0.3415],
        [ 0.2150,  0.1811,  0.7329, -0.0293,  0.1960, -0.0301,  0.0082,  0.0317]],
       grad_fn=<SliceBackward0>)

But first we need to apply the attention masking


In [402]:
mask_bool = mha.mask.bool()[:num_tokens, :num_tokens]

# apply the attention masking in place
attn_scores.masked_fill_(mask_bool, -torch.inf)
print("mask applied")

mask applied


In [403]:
attn_scores[0,0,:,:]

tensor([[-0.6037,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.9395, -1.3695,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.0748,  0.0079,  0.6271,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.1102, -0.4110,  0.4088, -0.1569,    -inf,    -inf,    -inf,    -inf],
        [-0.4095, -0.7016,  0.1857, -0.1982, -0.0505,    -inf,    -inf,    -inf],
        [-0.0080, -0.1904,  0.6200, -0.1120,  0.1329, -0.1724,    -inf,    -inf],
        [ 0.5833,  1.0433,  0.1107,  0.2706,  0.3087,  0.1986, -0.6728,    -inf],
        [ 0.2150,  0.1811,  0.7329, -0.0293,  0.1960, -0.0301,  0.0082,  0.0317]],
       grad_fn=<SliceBackward0>)

Now lets apply the softmax


In [404]:
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

In [405]:
attn_weights[0,0,:,:]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5535, 0.4465, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2888, 0.3010, 0.4102, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2419, 0.2081, 0.3136, 0.2363, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1812, 0.1566, 0.2440, 0.2014, 0.2168, 0.0000, 0.0000, 0.0000],
        [0.1606, 0.1466, 0.2199, 0.1525, 0.1724, 0.1480, 0.0000, 0.0000],
        [0.1630, 0.2052, 0.1287, 0.1394, 0.1421, 0.1345, 0.0870, 0.0000],
        [0.1273, 0.1252, 0.1650, 0.1127, 0.1261, 0.1127, 0.1148, 0.1162]],
       grad_fn=<SliceBackward0>)

Now dropout!


In [406]:
attn_weights = mha.dropout(attn_weights)

In [407]:
attn_weights[0,0,:,:]

tensor([[1.0526, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5827, 0.4700, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3040, 0.3168, 0.4318, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2547, 0.2191, 0.3301, 0.2488, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1648, 0.2569, 0.2120, 0.2282, 0.0000, 0.0000, 0.0000],
        [0.1691, 0.1544, 0.2315, 0.1605, 0.1814, 0.1558, 0.0000, 0.0000],
        [0.1716, 0.2160, 0.1355, 0.1468, 0.1496, 0.0000, 0.0916, 0.0000],
        [0.1340, 0.1318, 0.1737, 0.1186, 0.0000, 0.1186, 0.1209, 0.1223]],
       grad_fn=<SliceBackward0>)

Like before we now need to take the attention weights and combine them with our value vectors.\
\
As we can see, we are still broadcasting across the first two dimensions, so it is like we are performing the following:\
8x8 (attn_weights) @ 8x4 (values) -> 8x4 -> with batch dimension for all heads we have 2x4x8x4


In [409]:
attn_weights.shape, values.shape

(torch.Size([2, 4, 8, 8]), torch.Size([2, 4, 8, 4]))

but we want to use transpose to switch the context_length (in this case 8) and the dimension heads so we can join them together in a later step


In [410]:
# Shape: (b, num_tokens, num_heads, head_dim)
context_vec = (attn_weights @ values).transpose(1, 2) 

In [412]:
context_vec.shape

torch.Size([2, 8, 4, 4])

Why do we need `.continguous()` here?

- checkout this [post](https://stackoverflow.com/questions/48915810/what-does-contiguous-do-in-pytorch)


In [413]:
# Combine heads, where self.d_out = self.num_heads * self.head_dim
context_vec = context_vec.contiguous().view(b, num_tokens, mha.d_out)
context_vec = mha.o_proj(context_vec) # optional projection

In [414]:
context_vec.shape

torch.Size([2, 8, 16])

Finally, we reached the our output for the Multi-headed attention implementation!


#### **Lets Put it All together!**


In [438]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_in:int, d_out:int, context_length:int, dropout:float, num_heads=1, bias=False):
        super().__init__()
        assert(d_out % num_heads == 0), "The output dimension must be divisible by the number of heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # This reduces the projection dimention to match the desired output.

        # Define q, k, v. Remember these...
        self.W_query = nn.Linear(d_in, d_out, bias=bias)
        self.W_key = nn.Linear(d_in, d_out, bias=bias)
        self.W_value = nn.Linear(d_in, d_out, bias=bias)

        self.o_proj = nn.Linear(d_out, d_out) # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)

        # Create mask for decoder block
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
    

    def forward(self, x:torch.Tensor):
        b, num_tokens, d_in = x.shape

        keys:torch.Tensor = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries:torch.Tensor = self.W_query(x)
        values: torch.Tensor = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

       # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.size(-1)**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.o_proj(context_vec) # optional projection

        return context_vec

In [439]:
tokens, in_dim = 8, 6
inputs_v2 = torch.randn((tokens, in_dim))
# Create the batch inputs 
x_batch = torch.stack([inputs_v2, inputs_v2], dim=0)
print(f"Batch Shape: {x_batch.shape}")
print(f"There are {x_batch.shape[0]} exmaples of size {x_batch.shape[1]}x{x_batch.shape[2]} in the batch")
batch_size, context_length, d_in = x_batch.shape
d_out, num_heads = 16, 4

Batch Shape: torch.Size([2, 8, 6])
There are 2 exmaples of size 8x6 in the batch


In [440]:
mha = MultiHeadAttention(d_in, d_out, context_length, dropout=0.1, num_heads=4, bias=False)

In [441]:
x_batch.shape

torch.Size([2, 8, 6])

In [443]:
mha(x_batch).shape

torch.Size([2, 8, 16])

### Grouped Query Attention (GQA)

A more compute & parameter **efficient** implementation of attention. The idea is to reduce the number of final query groups by reducing the number of KV projections.


**Shared Buffers**

- class that is used alongside rope to ruse the attention mask, sin and cos computations each subsequen prediction that improves effeciency


In [ ]:
from rope import precompute_rope_params
class SharedBuffers:
    _buffers = {}

    @staticmethod
    def get_buffers(context_length, head_dim, rope_base, freq_config, dtype=torch.float32):
        # if rope config is not none, get the (mask, cos, sin) config values, otherwise pass none
        key = (context_length, head_dim, rope_base, tuple(freq_config.values()) if freq_config else freq_config, dtype)

        if key not in SharedBuffers._buffers:
            # Create or fetch the buffers
            mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
            cos, sin = precompute_rope_params(head_dim, rope_base, context_length, freq_config)
            if dtype is not None:
                cos = cos.to(dtype)
                sin = sin.to(dtype)
            SharedBuffers._buffers[key] = (mask, cos, sin)

        return SharedBuffers._buffers[key]

In [ ]:
class GroupedQueryAttention(nn.Module):
    def __init__(
            self, d_in, d_out, context_length, num_heads,
            num_kv_groups,       # NEW
            rope_base=10_000,    # NEW
            rope_config=None,    # NEW
            dtype=None
        ):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        assert num_heads % num_kv_groups == 0, "num_heads must be divisible by num_kv_groups"

        # Set the dimensions of the q, k, v queries
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        # Create the k and v weight matricies. 
        # Traditionally, the second dim is d_out. If num_kv_groups=1 we have Multi-Query, if num_kv_groups=num_head we have Multi-head attention
        self.W_key = nn.Linear(d_in, num_kv_groups * self.head_dim, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, num_kv_groups * self.head_dim, bias=False, dtype=dtype)
        self.num_kv_groups = num_kv_groups
        self.group_size = num_heads // num_kv_groups

        # query weights are the same as MHA
        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtye=dtype)
        
        # Fetch buffers using Shared buffers class
        mask, cos, sin = SharedBuffers.get_buffers(context_length, self.head_dim, rope_base, rope_config, dtype)

        self.register_buffer("mask", mask)
        self.register_buffer("cos", cos)
        self.register_buffer("sin", sin)

### Multi-headed Latent Attention

An efficient Attention implimenation with the DeepSeek-v2 model and futher validating with Deepseek-v3


In [ ]:
class MultiheadLatentAttention(nn.Module):
    def __init__(self, emb_dim, attn_heads, head_dim, compression_dim):
        super().__int__()

        # Down projection kv matrix

